In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
# 读取 URL 列表
df_urls = pd.read_csv('/home/liy/cqu_eie/DataEngineer/实验 Python程序设计-爬虫的实现-liy/url.csv')
urls = df_urls['url'].tolist()

In [3]:
# 设置 Chrome 浏览器选项
chrome_options = Options()
chrome_options.add_argument("--headless")  # 启用无头模式
chrome_options.add_argument("--disable-gpu")  # 禁用 GPU 加速
chrome_options.add_argument("--window-size=1920,1080") # 设置窗口大小
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36") # 设置用户代理
chrome_options.add_argument("--disable-extensions") # 禁用扩展
chrome_options.add_argument("--no-sandbox") # 禁用沙盒模式
chrome_options.add_argument("--disable-dev-shm-usage")  # 禁用禁用共享内存使用

In [4]:
# 定义线程任务函数
def scrape_product_info(index, url):
    start_time = time.time()  # 记录开始时间
    # print(f"开始处理产品 {index + 1}: {url}")

    # 启动 Chrome 浏览器（每个线程单独使用一个浏览器实例）
    service = Service('/usr/local/bin/chromedriver-linux64/chromedriver')  # 替换为你的 ChromeDriver 路径
    driver = webdriver.Chrome(service=service, options=chrome_options)

    try:
        driver.get(url)

        # 等待商品信息加载
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'jcdpl')))
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'MwTOW')))

        # 获取商品名称
        product_name = driver.find_element(By.CLASS_NAME, 'jcdpl').text

        # 获取价格信息
        price = driver.find_element(By.CLASS_NAME, 'MwTOW').text

        # 输出结果
        # print(f"产品名称: {product_name}")
        # print(f"价格: {price}")
    finally:
        driver.quit()

    # 记录结束时间
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"产品 {index + 1} 处理完毕，用时: {elapsed_time:.2f} 秒")

In [ ]:
max_workers = 15  # 设置最大线程数（根据系统性能调整）
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # 提交任务到线程池
    futures = [executor.submit(scrape_product_info, index, url) for index, url in enumerate(urls)]

    # 等待所有任务完成
    for future in futures:
        future.result()

In [ ]:
print("所有产品处理完毕！")

In [6]:
# # 保存到 CSV 文件
# results_df = pd.DataFrame(results)
# dir = "/home/liy/cqu_eie/DataEngineer/实验 Python程序设计-爬虫的实现-liy/test.csv"
# results_df.to_csv(dir, index=False, mode='a', header=not pd.io.common.file_exists(dir))